In [2]:
# -*- coding: utf-8 -*-
import numpy as np
import random
import matplotlib.pylab as pl
import matplotlib.animation as animation
%matplotlib qt5
Natom = 36 #原子数
NT = 1000 #最大时间步数
Tinit = 0.1 #初始温度
eps = 1.0 #势阱
x = np.zeros(Natom) #坐标
y = np.zeros(Natom)
vx = np.zeros(Natom) #速度
vy = np.zeros(Natom)
fx = np.zeros([Natom,2]) #力， 1）t时刻 2）t+dt时刻
fy = np.zeros([Natom,2])
L = int(1.0*Natom**0.5) #盒子边长

tt = np.arange(NT)  #时间
xc = np.zeros([Natom,NT+1]) #位置-时间
yc = np.zeros([Natom,NT+1]) 
EP = np.zeros(NT)  #势能-时间
EK = np.zeros(NT) #动能-时间
ET = np.zeros(NT) #总能-时间

def initialposvel(): #初始化
    i = -1
    for ix in range(L): #按格点摆放
        for iy in range(L):
            i = i + 1
            x[i] = ix
            y[i] = iy
            vx[i] = random.gauss(0,1)
            vy[i] = random.gauss(0,1)
            vx[i] = vx[i] * np.sqrt(Tinit)
            vy[i] = vy[i] * np.sqrt(Tinit)

def forces(t): #计算力
    r2cut = 9 #截断距离平方
    PE = 0.0 #势能置零
    for i in range(0,Natom): #力置零
        fx[i][t] = 0.0
        fy[i][t] = 0.0
    for i in range(0,Natom-1):
        for j in range(i+1,Natom):
            dx = x[i] - x[j] #计算距离
            dy = y[i] - y[j]
            if (dx > 0.5*L): #周期性边界条件
                dx = dx - L
            if (dx < -0.5*L):
                dx = dx + L

            if (dy > 0.5*L):
                dy = dy - L
            if (dy < -0.5*L):
                dy = dy + L

            r2 = dx*dx + dy*dy
            if(r2 < r2cut): #截断
                invr2 = 1.0/r2
                invr6 = invr2**3
                wij = 48*eps*invr2*invr6*(invr6-0.5)
                fijx = wij*dx
                fijy = wij*dy
                fx[i][t] = fx[i][t] + fijx
                fy[i][t] = fy[i][t] + fijy
                fx[j][t] = fx[j][t] - fijx
                fy[j][t] = fy[j][t] - fijy
                PE = PE + 4.0*eps*(invr6)*(invr6-1)
    return PE

def timevolution(): #时间演化
    t1 = 0
    t2 = 1
    h = 0.01 #时间步长
    hover2 = h/2.0
    initialposvel() #调用初始化
    PE = forces(t1) #计算力与势能
    for it in np.arange (NT): #时间循环
        if np.mod(it,100) == 0:
           print('it=',it)
        PE = forces(t1) #计算力与势能
        for i in range(0,Natom):

            x[i] = x[i] + h*(vx[i] + hover2*fx[i][t1]) #速度verlet更新位置
            y[i] = y[i] + h*(vy[i] + hover2*fy[i][t1])
            if x[i] <= 0: #周期边界
               x[i]=x[i] + L
            if x[i] > L:
               x[i]=x[i] - L
            if y[i] <= 0:
               y[i]=y[i] + L
            if y[i] > L:
               y[i]=y[i] - L
            xc[i][it] = x[i] #存储位置
            yc[i][it] = y[i] 

        PE = forces(t2) #计算势能与力
        KE = 0.0
        for i in range(0, Natom):
            vx[i] = vx[i] + hover2*(fx[i][t1] + fx[i][t2]) #速度verlet更新速度
            vy[i] = vy[i] + hover2*(fy[i][t1] + fy[i][t2])
            KE = KE + (vx[i]*vx[i] + vy[i]*vy[i])/2 #计算动能

        EP[it] = PE #存储势能
        EK[it] = KE #存储动能
        ET[it] = PE + KE #存储总能量
        
timevolution() 

def init():
    d.set_data([], [])
    return d,

def update_line(num, xc,yc,dot):
    dot.set_data(xc[:,num],yc[:,num])
    return dot,

fig1 = pl.figure()
d, = pl.plot([], [], 'ro',markersize=30)
pl.xlim(-0.5, 6.5)
pl.ylim(-0.5, 6.5)
pl.xlabel('X')
pl.ylabel('Y')
pl.title('MD')
dot_ani = animation.FuncAnimation(fig1, update_line, np.arange(1000),\
fargs=(xc,yc,d),interval=20, init_func=init, blit=True)

fig2 = pl.figure()
pl.plot(tt,EP,'k-')
pl.plot(tt,EK,'r-')
pl.plot(tt,ET,'b-')
pl.xlabel('time')
pl.ylabel('E')
pl.title('MD')
pl.show()

it= 0
it= 100
it= 200
it= 300
it= 400
it= 500
it= 600
it= 700
it= 800
it= 900
